<a href="https://colab.research.google.com/github/Ben-Ogega/Natural_Language_Processing/blob/main/sentimentAnalysis_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Import Libraries**

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import tensorflow_datasets as tfds

# **1. Loading the Dataset**

In [4]:
!pip install -q tfds-nightly tensorflow matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 15.7 MB/s eta 0:00:00


In [ ]:
#Find the List of available datasets
tfds.list_builders()

## The easiest way of loading a dataset is tfds.load. It will:

**Download the data and save it as tfrecord files.**

**Load the tfrecord and create the tf.data.Dataset.**

In [7]:
ds = tfds.load('mnist', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

<_PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [3]:
# Using the tfds.core.builder
builder = tfds.builder('mnist')
# 1. Create the tfrecord files (no-op if already exists)
builder.download_and_prepare()
# 2. Load the `tf.data.Dataset`
ds = builder.as_dataset(split='train', shuffle_files=True)
print(ds)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
<_PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


## **Iterate over a dataset**

In [4]:
ds = tfds.load('mnist', split='train')
ds = ds.take(1)  # Only take a single example

for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
  print(list(example.keys()))
  image = example["image"]
  label = example["label"]
  print(image.shape, label)

['image', 'label']
(28, 28, 1) tf.Tensor(4, shape=(), dtype=int64)


# **2. Process the data**

Benchmark your datasets
Benchmarking a dataset is a simple tfds.benchmark call on any iterable

(e.g. tf.data.Dataset, tfds.as_numpy,...).

In [5]:
ds = tfds.load('mnist', split='train')
ds = ds.batch(32).prefetch(1)

tfds.benchmark(ds, batch_size=32)
tfds.benchmark(ds, batch_size=32)  # Second epoch much faster due to auto-caching


************ Summary ************



  0%|          | 0/1875 [00:00<?, ?it/s]

Examples/sec (First included) 14014.59 ex/sec (total: 60032 ex, 4.28 sec)
Examples/sec (First only) 371.83 ex/sec (total: 32 ex, 0.09 sec)
Examples/sec (First excluded) 14294.31 ex/sec (total: 60000 ex, 4.20 sec)

************ Summary ************



  0%|          | 0/1875 [00:00<?, ?it/s]

Examples/sec (First included) 95505.40 ex/sec (total: 60032 ex, 0.63 sec)
Examples/sec (First only) 1097.95 ex/sec (total: 32 ex, 0.03 sec)
Examples/sec (First excluded) 100095.67 ex/sec (total: 60000 ex, 0.60 sec)


,duration,num_examples,avg
first+lasts,0.628572,60032,95505.401453
first,0.029145,32,1097.950158
lasts,0.599427,60000,100095.665264


# **3. Encode the data**

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

tfds.as_dataframe(ds.take(10), info)

# **4.Build the Model**

#**5. Train the Model**

# **6. Evaluate The Model**

# **7. Use the Model**